# CHIPPR

This notebook demonstrates the use of the Cosmological Hierarchical Inference with Probabilistic Photometric Redshifts (CHIPPR) package to estimate the redshift density function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from chippr import *

## Simulation

In [ ]:
tru_amps = np.array([0.20, 0.35, 0.55])
tru_means = np.array([0.5, 0.2, 0.75])
tru_sigmas = np.array([0.4, 0.2, 0.1])

tru_nz = gmix(tru_amps, tru_means, tru_sigmas)

In [ ]:
N = int(1e4)

tru_zs = tru_nz.sample(N)

plt.hist(tru_zs, bins=50, normed=1)
plt.xlabel(r'true $z$')
plt.ylabel(r'probability')

In [ ]:
params = {}

In [ ]:
lf_sigma = 0.05

params['sigma'] = lf_sigma

lf = likelihood(params)

obs_zs = np.array([lf.sample(z) for z in tru_zs])

plt.plot(tru_zs, obs_zs, 'k.', alpha=0.01)
plt.xlabel(r'true $z$')
plt.ylabel(r'peak $z$')

In [ ]:
params['int_pr'] = 'flat'

int_pr = interim_prior(params)

In [ ]:
post = posterior(lf, int_pr)
post.setup_bins()
data = post.make_data(obs_zs)

for n, z in enumerate(obs_zs[:10]):
    plt.plot(post.x_coarse, data[n], 'ko')
    plt.plot(post.x_fine, lf.evaluate_one(z, post.x_fine), 'k-')
    plt.show()

## Inference

In [ ]:
# data_loc = '../../../prob-z/nz/code/tests/sig2/data/logdata.csv'

# with open(data_loc,'rb') as data_file:
#     tuples = (line.split(None) for line in data_file)
#     data = [[float(pair[k]) for k in range(0,len(pair))] for pair in tuples]

In [ ]:
zs = post.binends
nz_intp = int_pr.evaluate(post.x_coarse)
z_posts = data

print(np.shape(zs), np.shape(nz_intp), np.shape(z_posts))

z_mids = (zs[1:]+zs[:-1])/2.
n_bins = len(z_mids)

In [ ]:
prior_sigma = 0.16
prior_var = np.eye(n_bins)
for b in range(n_bins):
    prior_var[b] = 1. * np.exp(-0.5 * (z_mids[b] - z_mids) ** 2 / prior_sigma ** 2)
l = 1.e-4
prior_var = prior_var+l*np.identity(n_bins)

In [ ]:
nz = z_dens(data, prior_var, truth=tru_nz, vb=True)

In [ ]:
help(z_dens)

In [ ]:
nz_mmap = nz.mmap()

In [ ]:
nz_mexp = nz.mexp()

In [ ]:
nz_stack = nz.stack()

In [ ]:
#nz_mmle = nz.infer(nz_lf)

In [ ]:
#nz_samps = nz.sample(nz_lf)

In [ ]:
nz.plot()